In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_weekly_demand_planning_store_sku
-- PARTITION BY calday CLUSTER BY dc_id, product_id 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH store_sku_list AS (
    SELECT 
        calday, store_id, product_id, dc_id, business_unit, replenishment_mode, mdq, is_problem_article, IF(replenishment_mode = 'FT', 14, 4) safety_stock,
        FIRST_VALUE(assortment_type) OVER (PARTITION BY dc_id, product_id ORDER BY 
            CASE
                WHEN assortment_type = 'Core 1' THEN 1
                WHEN assortment_type = 'HERO' THEN 2
                WHEN assortment_type = 'Core 2' THEN 3
                WHEN assortment_type = 'Core 3' THEN 4
                WHEN assortment_type = 'HI_BEAUTY' THEN 5
                ELSE 6
            END, assortment_type
        ) assortment_type
    FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his
    WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 14) AND business_unit IN ('1500', '2000') AND dc_id IS NOT NULL
),
_edq AS (
    SELECT date_add(date(calday), 1) AS calday, store_id, product_id, edq
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_edq_by_store_sku_current_his
    WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 15) 
),
_high_stock AS (
    SELECT
        date_add(date(calday), 1) AS calday,
        store_id, 
        product_id,
        concat_ws(', ', collect_list(reason)) AS reason
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_block_by_store_sku_current_his
    WHERE UPPER(reason) LIKE 'HIGH STOCK%' AND calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 15)
    GROUP BY 1,2,3
),
_block_promotion AS (
    SELECT 
        date_add(date(calday), 1) AS calday, store_id, IF(bonus_buy_profile = 'ZB01', buy_item_id, discount_product_id) product_id, 
        concat_ws(', ', collect_set(concat(bonus_buy_id, '-', bonus_buy_profile))) AS bonus_buy_id,
        SUM(block_sto) block_sto
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current_his
    WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 15) 
    GROUP BY 1,2,3
    HAVING block_sto = 1
),
dimension AS (
    SELECT 
        a.*, IFNULL(b.edq, 0) edq, c.reason block_reason, c1.bonus_buy_id,
        IF(a.replenishment_mode = 'BS' AND a.business_unit = '2000', 7, IF(a.replenishment_mode = 'FT' AND a.business_unit = '1500', d.dbo, 7)) targeted_dio_store,
        IFNULL(e.sell_rate, 0) sell_rate,
        IFNULL(e.sale_trend, 1) sale_trend,
        IFNULL(e.seasonality_index, 1) seasonality_index,
        IFNULL(f.closing_stock_qty, 0) closing_stock_qty,
        IFNULL(g.open_order, 0) open_order,
        h.mhu, i.pcb
    FROM store_sku_list a
    LEFT JOIN _edq b
        ON a.store_id = b.store_id AND a.product_id = b.product_id AND a.calday = b.calday
    LEFT JOIN _high_stock c
        ON a.store_id = c.store_id AND a.product_id = c.product_id AND a.calday = c.calday
    LEFT JOIN _block_promotion c1
        ON a.store_id = c1.store_id AND a.product_id = c1.product_id AND a.calday = c1.calday
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_dc_sku_list_his WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 15)) d
        ON a.dc_id = d.dc_id AND a.product_id = d.product_id AND a.calday = d.calday + INTERVAL 1 DAY
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dmt.f_corrected_sellrate WHERE calday >= date_sub(date(current_timestamp() + INTERVAL 7 HOURS), 15)) e 
    ON a.store_id = e.store_id AND a.product_id = e.product_id AND a.calday = date_add(e.calday, 1)
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_inventory_sapcar_daily WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 14)) f
        ON a.store_id = f.store_id AND a.product_id = f.product_id AND a.calday = f.calday
    LEFT JOIN (SELECT * FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.a_winare_dc_open_order_store_his WHERE calday >= DATE_SUB( DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)  , 15)) g
        ON a.store_id = g.store_id AND a.product_id = g.product_id AND a.calday = g.calday + INTERVAL 1 DAY
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_mhu_his h 
        ON a.product_id = h.product_id AND a.dc_id = h.dc_id AND a.business_unit = h.business_unit AND a.calday = h.calday + INTERVAL 1 DAY
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_pcb_his i 
        ON a.product_id = i.product_id AND a.dc_id = i.dc_id AND a.business_unit = i.business_unit AND a.calday = i.calday + INTERVAL 1 DAY
),
demand AS (
    SELECT
        *,
        IF(IFNULL(mdq, 0) = 0 OR is_problem_article = 'YES' OR block_reason IS NOT NULL, 1, 0) is_block,
        GREATEST(mdq + edq + (safety_stock + targeted_dio_store + 14) * sell_rate * sale_trend * seasonality_index - closing_stock_qty - open_order, 0) demand_store
    FROM dimension
),
round_mhu AS (
    SELECT
        * EXCEPT(demand_store),
        CEIL(
        CASE 
            WHEN mhu IS NULL OR mhu = 0 THEN NULL
            ELSE GREATEST(demand_store, 0) / mhu
        END
    ) * mhu AS demand_store
            FROM demand
),
picking AS (
    SELECT
        *,
        IF(IFNULL(pcb, 1) = 1, 'Piece', IF(demand_store / pcb = CAST(demand_store / pcb AS INT), 'Case', 'Piece')) picking_type
    FROM round_mhu
)
SELECT *
FROM picking;""")